In [1]:
import math as m
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import seaborn as sb
import random
import os
import nltk
from nltk.probability import FreqDist


import warnings
warnings.filterwarnings('ignore')

In [2]:
df_test = pd.read_csv('./StanceDataset/test_ingrid.csv')
df_train = pd.read_csv('./StanceDataset/train_ingrid.csv')

test_tweet = df_test['Tweet']
train_tweet = df_train['Tweet']

train_sentiment = df_train['Sentiment']
test_sentiment = df_test['Sentiment']

In [3]:
from nltk.corpus import wordnet as wn

In [4]:
liste = []

ListeTweet = df_train.Tweet.values.tolist()
ListSentiment = df_train.Sentiment.values.tolist()
dim = len(ListSentiment)
i = 0
#print(dim)

while i < dim:
    tmp = (ListeTweet[i], ListSentiment[i])
    liste.append(tmp)
    i += 1

#Liste de tuples (tweet,sentiment)
#print(liste)

# liste : Liste(tweet entier, sentiment)


tweets = []

for (words, sentiment) in liste:
    words_filtered = [e.lower() for e in words.split() if len(e) >= 2]  #1 pour avoir bcp de mot split
    tweets.append((words_filtered, sentiment))

def get_words_in_tweets(tweets):
    all_words = []
    for (words, sentiment) in tweets:
        all_words.extend(words)
    return all_words

def get_word_features(wordlist):
    wordlist = nltk.FreqDist(wordlist)
    word_features = wordlist.keys()
    return word_features

word_features = get_word_features(get_words_in_tweets(tweets))


tokens = get_words_in_tweets(tweets)
freq = nltk.FreqDist(tokens) 
#for key,val in freq.items(): 
   #print (str(key) + ':' + str(val))

In [18]:
X_train_posTweet = []
X_train_negTweet = []
X_train_otherTweet = []

X_test_posTweet = []
X_test_negTweet = []
X_test_otherTweet = []


dim = len(df_train)
dim = len(df_test)

for i in range(dim):
    if train_sentiment[i] == 'pos':
        X_train_posTweet.append(train_tweet[i])
    elif train_sentiment[i] == 'neg':
        X_train_negTweet.append(train_tweet[i])
    elif  train_sentiment[i] == 'other':
        X_train_otherTweet.append(train_tweet[i])

for i in range(dim):        
    if test_sentiment[i] == 'pos':
        X_test_posTweet.append(test_tweet[i])
    elif  test_sentiment[i] == 'neg':
        X_test_negTweet.append(test_tweet[i])
    elif  test_sentiment[i] == 'other':
        X_test_otherTweet.append(test_tweet[i])


In [34]:
X_train_wordPos = []
X_train_wordNeg = []
X_train_wordOther = []

X_test_wordPos = []
X_test_wordNeg = []
X_test_wordOther = []


for (words) in X_train_posTweet:
    words_filtered = [e.lower() for e in words.split() if len(e) >= 8]  #6 : longueur des mots que l'on prends
    X_train_wordPos.append(words_filtered)


for (words) in X_train_negTweet:
    words_filtered = [e.lower() for e in words.split() if len(e) >= 8]  #6 : longueur des mots que l'on prends
    X_train_wordNeg.append(words_filtered)
    

for (words) in X_train_otherTweet:
    words_filtered = [e.lower() for e in words.split() if len(e) >= 8]  #6 : longueur des mots que l'on prends
    X_train_wordOther.append(words_filtered)
    

    
for (words) in X_test_posTweet:
    words_filtered = [e.lower() for e in words.split() if len(e) >= 8]  #6 : longueur des mots que l'on prends
    X_test_wordPos.append(words_filtered)


for (words) in X_test_negTweet:
    words_filtered = [e.lower() for e in words.split() if len(e) >= 8]  #6 : longueur des mots que l'on prends
    X_test_wordNeg.append(words_filtered)
    

for (words) in X_test_otherTweet:
    words_filtered = [e.lower() for e in words.split() if len(e) >= 8]  #6 : longueur des mots que l'on prends
    X_test_wordOther.append(words_filtered)

In [40]:
import nltk.classify.util
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import names
 
    
#Featuresets are typically constructed using a "feature detector"
#(also known as a "feature extractor").  A feature detector is a
#function that takes a token (and sometimes information about its
#context) as its input, and returns a featureset describing that token.
#For example, the following feature detector converts a document
#(stored as a list of words) to a featureset describing the set of
#words included in the document:
 
  #  >>> # Define a feature detector function.
   # >>> def document_features(document):
   # ...     return dict([('contains-word(%s)' % w, True) for w in document])


#Bag of Words Feature Extraction
def word_feats(words):
    return dict([(word, True) for word in words])

 
positive_features_train = [(word_feats(pos), 'pos') for pos in X_train_wordPos]
negative_features_train = [(word_feats(neg), 'neg') for neg in X_train_wordNeg]
neutral_features_train = [(word_feats(other), 'other') for other in X_train_wordOther]

positive_features_test = [(word_feats(pos), 'pos') for pos in X_test_wordPos]
negative_features_test = [(word_feats(neg), 'neg') for neg in X_test_wordNeg]
neutral_features_test = [(word_feats(other), 'other') for other in X_test_wordOther]
 
train_set = negative_features_train + positive_features_train + neutral_features_train
test_set = negative_features_test + positive_features_test + neutral_features_test

classifier = NaiveBayesClassifier.train(train_set) 

#print(nltk.classify.accuracy(classifier,test_set))

accuracy = nltk.classify.util.accuracy(classifier, test_set)
print(accuracy * 100)
#classifier.show_most_informative_features()
# ACCURACY : taux de bonne prédiction

57.82208588957055


In [ ]:
#A learning algorithm is very useful for a classifier, here we will show you how to use the Naive Bayes and Maximum Entropy Model to train a NaiveBayes and Maxent Classifier, where Naive Bayes is the Generative Model and Maxent is Discriminative Model.

In [36]:
from nltk import MaxentClassifier
 
entropy_classifier = MaxentClassifier.train(train_set)
print(nltk.classify.accuracy(entropy_classifier, test_set))

  ==> Training (100 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1          -1.09861        0.351
             2          -0.56450        0.928
             3          -0.42090        0.946
             4          -0.34260        0.955
             5          -0.29257        0.960
             6          -0.25764        0.964
             7          -0.23176        0.964
             8          -0.21175        0.966
             9          -0.19579        0.966
            10          -0.18273        0.966
            11          -0.17183        0.967
            12          -0.16258        0.967
            13          -0.15462        0.968
            14          -0.14770        0.968
            15          -0.14161        0.968
            16          -0.13622        0.968
            17          -0.13140        0.968
            18          -0.12707        0.968
            19          -0.12315        0.968
 

In [38]:
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.svm import SVC, LinearSVC

In [39]:
#print("Original Naive Bayes Algo accuracy percent:", (nltk.classify.accuracy(classifier, test_set))*100)
#classifier.show_most_informative_features(15)

MNB_classifier = SklearnClassifier(MultinomialNB())
MNB_classifier.train(train_set)
print("MNB_classifier pourcentage de bonne prédiction :", (nltk.classify.accuracy(MNB_classifier, test_set))*100)
#The multinomial Naive Bayes classifier is suitable for classification with discrete features (e.g., word counts for text classification). The multinomial distribution normally requires integer feature counts. However, in practice, fractional counts such as tf-idf may also work.


BernoulliNB_classifier = SklearnClassifier(BernoulliNB())
BernoulliNB_classifier.train(train_set)
print("BernoulliNB_classifier pourcentage de bonne prédiction :", (nltk.classify.accuracy(BernoulliNB_classifier, test_set))*100)
#Naive Bayes classifier for multivariate Bernoulli models. Like MultinomialNB, this classifier is suitable for discrete data. The difference is that while MultinomialNB works with occurrence counts, BernoulliNB is designed for binary/boolean features.

LogisticRegression_classifier = SklearnClassifier(LogisticRegression())
LogisticRegression_classifier.train(train_set)
print("LogisticRegression_classifier pourcentage de bonne prédiction :", (nltk.classify.accuracy(LogisticRegression_classifier, test_set))*100)

SGDClassifier_classifier = SklearnClassifier(SGDClassifier())
SGDClassifier_classifier.train(train_set)
print("SGDClassifier_classifier pourcentage de bonne prédiction :", (nltk.classify.accuracy(SGDClassifier_classifier, test_set))*100)

SVC_classifier = SklearnClassifier(SVC())
SVC_classifier.train(train_set)
print("SVC_classifier pourcentage de bonne prédiction :", (nltk.classify.accuracy(SVC_classifier, test_set))*100)

LinearSVC_classifier = SklearnClassifier(LinearSVC())
LinearSVC_classifier.train(train_set)
print("LinearSVC_classifier pourcentage de bonne prédiction :", (nltk.classify.accuracy(LinearSVC_classifier, test_set))*100)


MNB_classifier pourcentage de bonne prédiction : 63.75255623721882
BernoulliNB_classifier pourcentage de bonne prédiction : 65.03067484662577
LogisticRegression_classifier pourcentage de bonne prédiction : 64.62167689161554
SGDClassifier_classifier pourcentage de bonne prédiction : 61.40081799591002
SVC_classifier pourcentage de bonne prédiction : 65.03067484662577
LinearSVC_classifier pourcentage de bonne prédiction : 60.88957055214724


## METHODE 2

In [ ]:
train_X = train_tweet
train_y = train_sentiment
 
test_X = test_tweet
test_y = test_sentiment

In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
from nltk import sent_tokenize, word_tokenize, pos_tag

In [ ]:
lemmatizer = WordNetLemmatizer()

In [ ]:
def penn_to_wn(tag):
    
    #Convert to simple Wordnet tags
    
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return None

In [ ]:
for i in range(len(train_y)):        
    if train_y[i] == 'pos':
        train_y[i] = 1
    elif  train_y[i] == 'neg':
        train_y[i] = 0
    elif  train_y[i] == 'other':
        train_y[i] = 2
        
for i in range(len(test_y)):        
    if test_y[i] == 'pos':
        test_y[i] = 1
    elif  test_y[i] == 'neg':
        test_y[i] = 0
    elif  test_y[i] == 'other':
        test_y[i] = 2

In [ ]:
def swn_polarity(text):

    #Return a sentiment polarity: 0 = negative, 1 = positive

 
    sentiment = 0.0
    tokens_count = 0
 

    raw_sentences = sent_tokenize(text)
    #print ("raw_sentences")
    #print (raw_sentences)
    
    for raw_sentence in raw_sentences:
        #print ("word_tokenize(raw_sentence)")
        #print (word_tokenize(raw_sentence))
        tagged_sentence = pos_tag(word_tokenize(raw_sentence))
        #print ("tagged_sentence")
        #print (tagged_sentence)
        
        for word, tag in tagged_sentence:
            wn_tag = penn_to_wn(tag)
            #print ("wn_tag")
            #print (wn_tag)
            if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV):
                continue
 
            lemma = lemmatizer.lemmatize(word, pos=wn_tag)
            #print ("lemma")
            #print (lemma)
            
            if not lemma:
                continue
 
            synsets = wn.synsets(lemma, pos=wn_tag)
            #print ("synsets")
            #print (synsets)
    
            if not synsets:
                continue
 
            # Take the first sense, the most common
            synset = synsets[0]
            swn_synset = swn.senti_synset(synset.name())
            #print ("swn_synset")
            #print (swn_synset)

            sentiment += swn_synset.pos_score() - swn_synset.neg_score()
            tokens_count += 1
            #print ("sentiment")
            #print (sentiment)
 

    # judgment call ? Default to positive or negative
    #if not tokens_count:
        #return 0
 
    # sum greater than 0 => positive sentiment
    #if sentiment >= 0:
        #return 1
 
    # negative sentiment
    #return 0
    if sentiment < 0:
        return 0
    elif sentiment > 0:
        return 1
    elif sentiment == 0:
        return 2
 
#print (swn_polarity(train_X[0]), train_y[0])

dim = len(train_X)
Nberreur = 0

for j in range(dim):
    if swn_polarity(train_X[j]) != train_y[j]:
        Nberreur=Nberreur+1
    j=j+1 

taux_erreur = 1-(dim-Nberreur)/dim

print(taux_erreur)

## METHODE 3

In [ ]:
sentiment_data = zip(df_train["Tweet"], df_train["Sentiment"])